In [2]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [16]:
import pandas as pd 

data = pd.read_csv("data.csv",index_col=0)
data.size

26919

In [ ]:
import requests
from PIL import Image
from os import listdir

present = listdir("kanjing")

for i in range(data.shape[0]):
    row = data.loc[i]
    kanji = row.kanji
    if kanji + ".png" in present:
        continue
    print(i, kanji)
    response = requests.get(f"https://kanji.sljfaq.org/kanjivg/memory.cgi?k={kanji}&s=5&n=0.4&l=4", stream=True)
    response.raw.decode_content = True
    Image.open(response.raw).save("kanjing/" + kanji + ".png")
print("done")

In [24]:
from PIL import Image
from os import listdir
import requests

def finditem(obj, key):
    if key in obj: return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = finditem(v, key)
            if item is not None:
                return item
def get_if_not(keyword) -> bool:
    present = listdir("keywordng")

    if keyword + ".png" in present:
        return True
    print(keyword)
    try:
        response = requests.get(f"https://api.langeek.co/v1/cs/en/word/?term={keyword}&filter=,inCategory,photo").json()
        keyword_image = finditem(response[0], "photo")
        if keyword_image:
            response = requests.get(keyword_image, stream=True)
            response.raw.decode_content = True
            Image.open(response.raw).save("keywordng/" + keyword + ".png")
            return True
    except Exception as e:
        # print(e)
        return False

    return False

In [15]:
import random 
import markdown
from ipywidgets import *

# (294, 546)
bucket = list(range(294))
# bucket = list(range(294, 546))
# bucket = list(range(546, 2229))

btn_show = Button(icon="check", button_style="primary", 
                  layout=Layout(width="60px", height="30px", margin="1rem 5rem auto auto"))
keyword_label = Label(value="", style=dict(text_color="#eeeeee", font_size="30px"), 
                  layout=Layout(margin="1rem 0 1rem 0"))
kanji_image = Image(format="png", height=500, width=500)
kanji_image.set_value_from_file("kanjing/1.png")
story_label = HTML(value="", style=dict(text_color="#eeeeee", font_size="16px"))

box = VBox((HBox((keyword_label, btn_show)), kanji_image, story_label))

next_question = False
# random.shuffle(bucket)

def question(arg=None):
    global bucket, next_question, keyword_label
    next_question = not next_question

    row = data.loc[bucket[0]]
    keyword, kanji, story = row.keyword, row.kanji, row.koohi1.split(":")[2]

    if next_question:
        btn_show.button_style = "primary"
        keyword_label.value = keyword
        kanji_image.set_value_from_file("kanjing/1.png")
        story_label.value = ""
    else:
        btn_show.button_style = "success"
        kanji_image.set_value_from_file(f"kanjing/{kanji}.png")
        story_label.value = markdown.markdown(story)
        bucket.pop(0)

btn_show.on_click(question)

box